Setup & Load Previous Data

In [ ]:
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from config import Config
from utils import setup_logging, validate_sequence, clean_sequence, save_checkpoint

# Import Biopython for sequence analysis
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Alternative GC content calculation
def calculate_gc_content(sequence):
    """Calculate GC content without Bio.SeqUtils"""
    if not sequence:
        return 0.0
    seq_upper = sequence.upper()
    gc_count = seq_upper.count('G') + seq_upper.count('C')
    return (gc_count / len(sequence)) * 100 if len(sequence) > 0 else 0.0

# Setup logging
logger = setup_logging()
logger.info("Starting complete data processing notebook")

# Load previous results
try:
    with open('../data/processed/exploration_results.json', 'r') as f:
        exploration_results = json.load(f)
    print("Previous results loaded successfully")
except FileNotFoundError:
    print("Warning: Previous exploration results not found")
    exploration_results = {}

# Find and load the most recent metadata file
import glob
metadata_files = glob.glob('../data/processed/biston_betularia_global_enhanced_*.csv')
if metadata_files:
    latest_metadata_file = max(metadata_files)
    df_original = pd.read_csv(latest_metadata_file)
    print(f"Loaded metadata: {latest_metadata_file}")
    print(f"Records loaded: {len(df_original)}")
    print(f"Columns: {list(df_original.columns)}")
else:
    print("Error: No metadata files found. Please run Notebook 2 first.")
    df_original = pd.DataFrame()

2025-06-01 13:21:22,584 - INFO - Starting complete data processing notebook


Previous results loaded successfully
Error: No metadata files found. Please run Notebook 2 first.


QUALITY CONTROL FUNCTION

In [127]:
def comprehensive_sequence_qc(df):
    """Comprehensive quality control untuk sequences"""
    if df.empty:
        print("No data to process")
        return df, {}
    
    initial_count = len(df)
    qc_stats = {
        'initial_sequences': initial_count,
        'removed_duplicates': 0,
        'removed_short': 0,
        'removed_long': 0,
        'removed_invalid_chars': 0,
        'removed_high_ambiguous': 0,
        'final_sequences': 0
    }
    
    print("SEQUENCE QUALITY CONTROL")
    print("="*40)
    
    # 1. Remove duplicates based on sequence
    print(f"1. Checking for duplicate sequences...")
    before_dedup = len(df)
    df_clean = df.drop_duplicates(subset=['sequence'], keep='first').copy()
    qc_stats['removed_duplicates'] = before_dedup - len(df_clean)
    print(f"   Removed {qc_stats['removed_duplicates']} duplicate sequences")
    
    # 2. Length filtering
    print(f"2. Filtering by sequence length ({Config.MIN_SEQUENCE_LENGTH}-{Config.MAX_SEQUENCE_LENGTH} bp)...")
    before_length = len(df_clean)
    df_clean = df_clean[
        (df_clean['sequence_length'] >= Config.MIN_SEQUENCE_LENGTH) &
        (df_clean['sequence_length'] <= Config.MAX_SEQUENCE_LENGTH)
    ].copy()
    qc_stats['removed_short'] = before_length - len(df_clean)
    print(f"   Removed {qc_stats['removed_short']} sequences outside length range")
    
    # 3. Clean sequences and check for invalid characters
    print(f"3. Cleaning sequences and removing invalid characters...")
    def check_and_clean_sequence(seq):
        cleaned = clean_sequence(seq)
        valid_chars = set('ATGCNRYSWKMBDHV')
        return cleaned if set(cleaned.upper()).issubset(valid_chars) else None
    
    before_clean = len(df_clean)
    df_clean['sequence_cleaned'] = df_clean['sequence'].apply(check_and_clean_sequence)
    df_clean = df_clean.dropna(subset=['sequence_cleaned']).copy()
    qc_stats['removed_invalid_chars'] = before_clean - len(df_clean)
    print(f"   Removed {qc_stats['removed_invalid_chars']} sequences with invalid characters")
    
    # 4. Remove sequences with too many ambiguous bases
    print(f"4. Filtering sequences with >10% ambiguous bases...")
    def calculate_ambiguous_percentage(seq):
        ambiguous_count = sum(1 for char in seq.upper() if char in 'NRYSWKMBDHV')
        return ambiguous_count / len(seq) if len(seq) > 0 else 1.0
    
    before_ambiguous = len(df_clean)
    df_clean['ambiguous_percentage'] = df_clean['sequence_cleaned'].apply(calculate_ambiguous_percentage)
    df_clean = df_clean[df_clean['ambiguous_percentage'] <= 0.1].copy()
    qc_stats['removed_high_ambiguous'] = before_ambiguous - len(df_clean)
    print(f"   Removed {qc_stats['removed_high_ambiguous']} sequences with >10% ambiguous bases")
    
    # 5. Update sequence column with cleaned version
    df_clean['sequence'] = df_clean['sequence_cleaned']
    df_clean['sequence_length'] = df_clean['sequence'].str.len()
    
    qc_stats['final_sequences'] = len(df_clean)
    
    print(f"\nQUALITY CONTROL SUMMARY:")
    print(f"Initial sequences: {qc_stats['initial_sequences']}")
    print(f"Final sequences: {qc_stats['final_sequences']}")
    print(f"Retention rate: {qc_stats['final_sequences']/qc_stats['initial_sequences']*100:.1f}%")
    
    return df_clean, qc_stats

DIVERSITY ANALYSIS FUNCTION

In [128]:
def analyze_sequence_diversity(df):
    """Analyze genetic diversity dalam dataset"""
    if df.empty:
        return {}
    
    print("\nSEQUENCE DIVERSITY ANALYSIS")
    print("="*40)
    
    sequences = df['sequence'].tolist()
    diversity_stats = {}
    
    # 1. Basic statistics
    diversity_stats['total_sequences'] = len(sequences)
    diversity_stats['unique_sequences'] = len(set(sequences))
    diversity_stats['sequence_identity'] = diversity_stats['unique_sequences'] / diversity_stats['total_sequences']
    
    print(f"Total sequences: {diversity_stats['total_sequences']}")
    print(f"Unique sequences: {diversity_stats['unique_sequences']}")
    print(f"Sequence diversity: {diversity_stats['sequence_identity']:.3f}")
    
    # 2. Length distribution
    lengths = df['sequence_length'].tolist()
    diversity_stats['mean_length'] = np.mean(lengths)
    diversity_stats['length_std'] = np.std(lengths)
    diversity_stats['min_length'] = min(lengths)
    diversity_stats['max_length'] = max(lengths)
    
    print(f"\nLength statistics:")
    print(f"Mean length: {diversity_stats['mean_length']:.1f} ± {diversity_stats['length_std']:.1f} bp")
    print(f"Length range: {diversity_stats['min_length']}-{diversity_stats['max_length']} bp")
    
    # 3. GC content analysis
    gc_contents = [calculate_gc_content(seq) for seq in sequences]
    diversity_stats['mean_gc'] = np.mean(gc_contents)
    diversity_stats['gc_std'] = np.std(gc_contents)
    diversity_stats['min_gc'] = min(gc_contents)
    diversity_stats['max_gc'] = max(gc_contents)
    
    print(f"\nGC content statistics:")
    print(f"Mean GC content: {diversity_stats['mean_gc']:.1f} ± {diversity_stats['gc_std']:.1f}%")
    print(f"GC range: {diversity_stats['min_gc']:.1f}-{diversity_stats['max_gc']:.1f}%")
    
    # 4. Nucleotide composition
    all_sequence = ''.join(sequences).upper()
    total_bases = len(all_sequence)
    base_counts = Counter(all_sequence)
    
    print(f"\nNucleotide composition:")
    for base in ['A', 'T', 'G', 'C']:
        count = base_counts.get(base, 0)
        percentage = count / total_bases * 100 if total_bases > 0 else 0
        print(f"{base}: {count:,} ({percentage:.1f}%)")
        diversity_stats[f'{base.lower()}_percentage'] = percentage
    
    return diversity_stats

HAPLOTYPE IDENTIFICATION FUNCTION

In [129]:
def identify_haplotypes_simple(df, similarity_threshold=0.97):
    """Simple haplotype identification"""
    if df.empty:
        return df, {}, []
    
    print("\nHAPLOTYPE IDENTIFICATION")
    print("="*40)
    print(f"Using similarity threshold: {similarity_threshold}")
    
    sequences = df['sequence'].tolist()
    unique_sequences = list(set(sequences))
    
    print(f"Total sequences: {len(sequences)}")
    print(f"Unique sequences: {len(unique_sequences)}")
    
    # Simple clustering berdasarkan exact matches
    haplotype_groups = []
    sequence_to_haplotype = {}
    
    # Start dengan exact matches
    sequence_counts = Counter(sequences)
    haplotype_id = 1
    
    for seq, count in sequence_counts.most_common():
        if seq not in sequence_to_haplotype:
            haplotype_name = f"Hap_{haplotype_id:03d}"
            sequence_to_haplotype[seq] = haplotype_name
            haplotype_groups.append({
                'haplotype_id': haplotype_name,
                'representative_sequence': seq,
                'sequence_count': count,
                'frequency': count / len(sequences)
            })
            haplotype_id += 1
    
    # Assign haplotypes to dataframe
    df_haplotypes = df.copy()
    df_haplotypes['haplotype_id'] = df_haplotypes['sequence'].map(sequence_to_haplotype)
    
    # Calculate haplotype statistics
    haplotype_stats = {
        'total_haplotypes': len(haplotype_groups),
        'most_common_haplotype': haplotype_groups[0]['haplotype_id'] if haplotype_groups else None,
        'most_common_frequency': haplotype_groups[0]['frequency'] if haplotype_groups else 0,
        'singleton_haplotypes': sum(1 for h in haplotype_groups if h['sequence_count'] == 1),
        'haplotype_diversity': len(haplotype_groups) / len(sequences) if len(sequences) > 0 else 0
    }
    
    print(f"\nHaplotype identification results:")
    print(f"Total haplotypes identified: {haplotype_stats['total_haplotypes']}")
    print(f"Most common haplotype: {haplotype_stats['most_common_haplotype']} ({haplotype_stats['most_common_frequency']:.3f})")
    print(f"Singleton haplotypes: {haplotype_stats['singleton_haplotypes']}")
    print(f"Haplotype diversity: {haplotype_stats['haplotype_diversity']:.3f}")
    
    # Print top 10 haplotypes
    print(f"\nTop 10 most common haplotypes:")
    for i, hap in enumerate(haplotype_groups[:10]):
        print(f"{i+1:2d}. {hap['haplotype_id']}: {hap['sequence_count']:3d} sequences ({hap['frequency']:.3f})")
    
    return df_haplotypes, haplotype_stats, haplotype_groups

EXPORT FUNCTION

In [130]:
def export_final_dataset(df, haplotype_groups, qc_stats, diversity_stats, haplotype_stats):
    """Export final processed dataset untuk ABM simulation"""
    if df.empty:
        print("No data to export")
        return None, None, None
    
    # Create final directory if it doesn't exist
    os.makedirs("../data/final", exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. Export main dataset
    final_dataset_file = f"../data/final/biston_betularia_final_dataset_{timestamp}.csv"
    
    # Select important columns untuk ABM
    export_columns = [
        'accession_id', 'haplotype_id', 'sequence', 'sequence_length',
        'country', 'collection_date', 'lat_lon', 'collected_by',
        'organism', 'description'
    ]
    
    # Only include columns that exist
    available_columns = [col for col in export_columns if col in df.columns]
    df_export = df[available_columns].copy()
    
    # Add GC content
    df_export['gc_content'] = df_export['sequence'].apply(calculate_gc_content)
    
    df_export.to_csv(final_dataset_file, index=False)
    print(f"\nFinal dataset exported: {final_dataset_file}")
    print(f"Records: {len(df_export)}")
    print(f"Columns: {list(df_export.columns)}")
    
    # 2. Export haplotype summary
    haplotype_summary_file = f"../data/final/haplotype_summary_{timestamp}.csv"
    haplotype_df = pd.DataFrame(haplotype_groups)
    haplotype_df.to_csv(haplotype_summary_file, index=False)
    print(f"Haplotype summary exported: {haplotype_summary_file}")
    
    # 3. Export analysis summary
    analysis_summary = {
        'processing_date': datetime.now().isoformat(),
        'input_sequences': qc_stats.get('initial_sequences', 0),
        'final_sequences': qc_stats.get('final_sequences', 0),
        'retention_rate': qc_stats.get('final_sequences', 0) / qc_stats.get('initial_sequences', 1),
        'unique_sequences': diversity_stats.get('unique_sequences', 0),
        'total_haplotypes': haplotype_stats.get('total_haplotypes', 0),
        'haplotype_diversity': haplotype_stats.get('haplotype_diversity', 0),
        'mean_sequence_length': diversity_stats.get('mean_length', 0),
        'mean_gc_content': diversity_stats.get('mean_gc', 0),
        'quality_control': qc_stats,
        'diversity_analysis': diversity_stats,
        'haplotype_analysis': haplotype_stats
    }
    
    summary_file = f"../data/final/analysis_summary_{timestamp}.json"
    with open(summary_file, 'w') as f:
        json.dump(analysis_summary, f, indent=2)
    print(f"Analysis summary exported: {summary_file}")
    
    return final_dataset_file, haplotype_summary_file, summary_file

VisualizationMAIN PROCESSING PIPELINE

In [131]:
def run_complete_processing():
    """Run the complete processing pipeline"""
    
    if df_original.empty:
        print("ERROR: No input data available. Please run Notebook 2 first.")
        return
    
    print(f"Starting processing pipeline with {len(df_original)} sequences")
    print("="*60)
    
    # Step 1: Quality Control
    df_clean, qc_statistics = comprehensive_sequence_qc(df_original)
    
    if df_clean.empty:
        print("ERROR: No sequences passed quality control")
        return
    
    # Step 2: Diversity Analysis
    diversity_analysis = analyze_sequence_diversity(df_clean)
    
    # Step 3: Haplotype Identification
    df_haplotypes, haplotype_statistics, haplotype_groups = identify_haplotypes_simple(
        df_clean, 
        similarity_threshold=Config.SIMILARITY_THRESHOLD
    )
    
    # Step 4: Export Results
    final_files = export_final_dataset(
        df_haplotypes, 
        haplotype_groups, 
        qc_statistics, 
        diversity_analysis, 
        haplotype_statistics
    )
    
    # Step 5: Final Summary
    print(f"\n{'='*60}")
    print("DATA PROCESSING COMPLETE")
    print(f"{'='*60}")
    print(f"Processing completed successfully!")
    print(f"Final dataset ready for ABM simulation")
    
    print(f"\nFINAL RESULTS SUMMARY:")
    print(f"Total sequences processed: {qc_statistics['final_sequences']}")
    print(f"Unique haplotypes identified: {haplotype_statistics['total_haplotypes']}")
    print(f"Haplotype diversity: {haplotype_statistics['haplotype_diversity']:.3f}")
    print(f"Mean sequence length: {diversity_analysis['mean_length']:.1f} bp")
    print(f"Mean GC content: {diversity_analysis['mean_gc']:.1f}%")
    print(f"Data retention rate: {qc_statistics['final_sequences']/qc_statistics['initial_sequences']*100:.1f}%")
    
    return df_haplotypes, haplotype_groups, qc_statistics, diversity_analysis, haplotype_statistics

RUN THE PIPELINE

In [132]:
if not df_original.empty:
    results = run_complete_processing()
    if results:
        df_haplotypes, haplotype_groups, qc_statistics, diversity_analysis, haplotype_statistics = results
        print("\nAll variables successfully created and available for further analysis!")
    else:
        print("Processing failed - check input data and error messages above")
else:
    print("No input data available - please run Notebook 2 first to fetch data from NCBI")

No input data available - please run Notebook 2 first to fetch data from NCBI
